In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/data/flask_deploy_model

/content/drive/MyDrive/Colab Notebooks/data/flask_deploy_model


**Creating csv file containing name of videos in dataset with their subfolder to read it later**

In [ ]:
import os
import numpy as np
import csv 

users = os.listdir("/content/drive/MyDrive/videos_ppg/videos")
users = list(filter(lambda x: os.path.isdir(os.path.join("/content/drive/MyDrive/videos_ppg/videos", x)), users))
print(users)
list_users = []
fields=['label','video']
filename = "data_records.csv"
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerow(fields) 

for i, user in enumerate(sorted(users)):
    user_fold = os.path.join("/content/drive/MyDrive/videos_ppg/videos/", user)
    #print(user_fold)
    user_files = os.listdir(user_fold)
    list_users.append(user_files)
    
arr_users = np.array(list_users)

for i in range (4):
    for j in range(len(arr_users[i])):
        split = arr_users[i][j].split('_')
        name = split[0]
        video = arr_users[i][j]
        row = [name,video]
        with open(filename, 'a') as csvfile: 
            # creating a csv writer object 
            csvwriter = csv.writer(csvfile) 
            csvwriter.writerow(row) 

print(arr_users[0])
print(len(arr_users[0]))





sent = 'abdullah_S1'
stri = sent.split('_')
print(stri)
str1 = stri[0]

['abdul', 'arham', 'arqam', 'ghazanfar']
['abdul_s1.mp4' 'abdul_s2.mp4' 'abdul_s4.mp4' 'abdul_s3.mp4'
 'abdul_s6.mp4' 'abdul_s5.mp4']
6
['abdullah', 'S1']


In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
#from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import cv2
# Ignore warnings
import warnings
from sklearn.metrics import accuracy_score as acc

from sklearn.metrics import confusion_matrix

from sklearn.metrics import matthews_corrcoef as mcor

from sklearn.metrics import precision_score as precision

from sklearn.metrics import recall_score as recall

from torch.utils.data import DataLoader

from torch.utils.tensorboard import SummaryWriter

warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [ ]:
import torchvision
print(torchvision.__version__)

0.14.0+cu116


**Creating Custom Dataset**

 ---


In [ ]:
class PpgFramesDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        
       
        self.ppg_frames = pd.read_csv(csv_file)
        print(self.ppg_frames)
        self.root_dir = root_dir
        video_name = os.path.join("/content/drive/MyDrive/videos_ppg/videos/"+self.ppg_frames.iloc[0, 0],self.ppg_frames.iloc[0, 1])
        print(video_name)
        self.transform = transform

    def __len__(self):
        return len(self.ppg_frames)

    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()

        video_name = os.path.join("/content/drive/MyDrive/videos_ppg/videos/"+self.ppg_frames.iloc[idx, 0],self.ppg_frames.iloc[idx, 1])
        #image = io.imread(img_name)
        video = cv2.VideoCapture(video_name)
        
        i=0
        data=None
        while(video.isOpened()):
            ret, frame = video.read()
            if ret == False:
                break
            #print(frame.shape)
            frame=cv2.resize(frame, (256,224))
            #input_tensor = preprocess(frame)
            #print(frame.shape)
            if(i==0):
               
                data=frame
            else:
                data=np.concatenate((data,frame),axis=2)
            i+=1
        video.release()
        cv2.destroyAllWindows()
        ppgFrames = self.ppg_frames.iloc[idx, 1:]
        print(ppgFrames)
        ppgFrames = np.array([ppgFrames])
        #ppgFrames = ppgFrames.astype('float').reshape(-1, 2)
        sample = {'image': data, 'labels': ppgFrames}

        if self.transform:
            sample = self.transform(sample)

        return sample



**Function to calculate accuracy of model**

In [ ]:
def accuracy(outputs, trues):
    
    ### Converting preds to 0 or 1
    outputs = [1 if outputs[i] >= 0.5 else 0 for i in range(len(outputs))]
    print(outputs)
    
    ### Calculating accuracy by comparing predictions with true labels
    acc = [1 if outputs[i] == trues[i] else 0 for i in range(len(outputs))]
    
    ### Summing over all correct predictions
    acc = np.sum(acc) / len(outputs)
    
    return (acc * 100)

**Resnet Pytorch Model**

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import pickle
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)

from torchvision import transforms
model.eval()
#pickle.dump(model,open('model_pkl','wb'))


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:

with open("/content/drive/MyDrive/videos_ppg/persons2.txt", "r") as f:
  categories = [s.strip() for s in f.readlines()]

print(categories)
# a temporary list to store the string labels
trues = []
outputs = []

# dictionary that maps integer to its string value 
label_dict = {}

# list to store integer labels 
int_labels = []

for i in range(len(categories)):
    label_dict[i] = categories[i]
    int_labels.append(i)


criterion = nn.SmoothL1Loss()

def tens(sample):
  size = sample.shape[2]
 
  print('size',size)
  model.conv1 = nn.Conv2d(size, 64, kernel_size=7, stride=2, padding=3,bias=False)
  model.fc = nn.Linear(512,24)
  preprocess = transforms.Compose([
      transforms.ToTensor(),

      transforms.Normalize(mean=np.full((size), 0.485, dtype=float), std=np.full((size), 0.229, dtype=float)),
  ])

  input_tensor = preprocess(sample)
  print('input_tenor',input_tensor)
  return input_tensor

ppg_dataset = PpgFramesDataset(csv_file='data_records.csv',
                                    root_dir="")

fig = plt.figure()
print('length',len(ppg_dataset))
#pickle.dump(model,open('model_pkl','wb'))


['abdul', 'abdul', 'abdul', 'abdul', 'abdul', 'abdul', 'arham', 'arham', 'arham', 'arham', 'arham', 'arham', 'arqam', 'arqam', 'arqam', 'arqam', 'arqam', 'ghazanfar', 'ghazanfar', 'ghazanfar', 'ghazanfar', 'ghazanfar', 'ghazanfar', '']
        label             video
0       abdul      abdul_s1.mp4
1       abdul      abdul_s2.mp4
2       abdul      abdul_s4.mp4
3       abdul      abdul_s3.mp4
4       abdul      abdul_s6.mp4
5       abdul      abdul_s5.mp4
6       arham      arham_s4.mp4
7       arham      arham_s1.mp4
8       arham      arham_s2.mp4
9       arham      arham_s3.mp4
10      arham      arham_s6.mp4
11      arham      arham_s5.mp4
12      arqam      arqam_s2.mp4
13      arqam      arqam_s3.mp4
14      arqam      arqam_s1.mp4
15      arqam      arqam_s6.mp4
16      arqam      arqam_s5.mp4
17      arqam      arqam_s4.mp4
18  ghazanfar  ghazanfar_s3.mp4
19  ghazanfar  ghazanfar_s2.mp4
20  ghazanfar  ghazanfar_s1.mp4
21  ghazanfar  ghazanfar_s6.mp4
22  ghazanfar  ghazanfar_s5.

<Figure size 432x288 with 0 Axes>

In [ ]:
for i in range(len(ppg_dataset)):
    sample = ppg_dataset[i]
   
    print(i, sample['image'].shape, sample['labels'].shape)

    
    #size = sample['image'].shape[2]
    print(sample['image'].shape[2])
    current_name = sample['labels'][0][0].split('_')[0]
    print(current_name)
    input_tensor = tens(sample['image'])

    # create a mini-batch as expected by the model
    input_batch = input_tensor.unsqueeze(0)
    
    #move the input and model to GPU for speed if available
    if torch.cuda.is_available():
      input_batch = input_batch.to('cuda')
      model.to('cuda')
    print(input_batch.shape)
    with torch.no_grad():
      output = model(input_batch)


    print(output[0])
    labels = torch.tensor(int_labels)
    loss = criterion(output[0],labels)
    print(loss)

    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    print('probabilities',probabilities.size(0))
  
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    print(torch.topk(probabilities, 5))
    if categories[top5_catid[0]] == current_name:
      trues.append(1)
      print('true')
    else:
      trues.append(0)
      print('false')
    outputs.append(top5_prob[0].item())
    #print(top5_catid)
    for i in range(top5_prob.size(0)):
      print(categories[top5_catid[i]], top5_prob[i].item())

print('accuracy: ',accuracy(outputs,trues))
pickle.dump(model,open('model_pkl','wb'))

video    abdul_s1.mp4
Name: 0, dtype: object
0 (224, 256, 5691) (1, 1)
5691
abdul
size 5691
input_tenor tensor([[[-2.1008, -2.1008, -2.0837,  ..., -2.0837, -2.0665, -2.0837],
         [-2.0837, -2.1008, -2.0837,  ..., -2.0837, -2.0837, -2.0837],
         [-2.0837, -2.1008, -2.1008,  ..., -2.1008, -2.0494, -2.1008],
         ...,
         [-2.1179, -2.1008, -2.1008,  ..., -2.0665, -2.0837, -2.0837],
         [-2.1179, -2.1008, -2.1179,  ..., -2.0837, -2.0665, -2.0665],
         [-2.1179, -2.1179, -2.1179,  ..., -2.0837, -2.0837, -2.0837]],

        [[-1.5014, -1.5014, -1.4843,  ..., -1.8097, -1.7925, -1.7925],
         [-1.4843, -1.5014, -1.4843,  ..., -1.8097, -1.8097, -1.7925],
         [-1.4843, -1.5014, -1.5014,  ..., -1.8268, -1.7754, -1.8097],
         ...,
         [-1.5185, -1.5014, -1.5014,  ..., -1.7925, -1.8097, -1.8097],
         [-1.5185, -1.5014, -1.5185,  ..., -1.8097, -1.7925, -1.7925],
         [-1.5185, -1.5357, -1.5357,  ..., -1.8097, -1.8097, -1.8097]],

        [[ 1

Flask


In [ ]:
!pip install flask
!pip install flask-ngrok
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15982 sha256=844ff27f4ffa2c32ebbb760fe57514bee004326ba12284105324f91141e6c56b
  Stored in directory: /root/.cache/pip/wheels/5e/0a/51/8cb053ccd84481dd3233eba4cdb608bc7a885fd8ca418c0806
Successfully built pyngrok


Write your authtoken below

In [ ]:
!ngrok authtoken 'here'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


**Flask Application**



We deployed code to server using flask and receive video from flutter and run the model on video received and send result back to flutter.


In [24]:
import numpy as np
from flask_ngrok import run_with_ngrok
from flask import Flask,request,jsonify,render_template,redirect
import pickle
from werkzeug.utils import secure_filename
import urllib.request
from pathlib import Path
import tempfile

app = Flask(__name__, template_folder='templates')

run_with_ngrok(app)

model1 = pickle.load(open('model_pkl','rb'))


ind =0
@app.route('/')
def upload_form():
	return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_video():
	
	if 'file' not in request.files:
		return redirect(request.url)

	upload_file = request.files['file']
	if upload_file.filename == '':
		return redirect(request.url)
	else:
		filename = secure_filename(upload_file.filename)
		current_name = filename.split('_')[0]
		with tempfile.TemporaryDirectory() as td:
			temp_filename = Path(td) / 'uploaded_video'
			upload_file.save(temp_filename)
	 
		
			cap = cv2.VideoCapture(str(temp_filename))
			i=0
			data=None
			while(cap.isOpened()):
				ret, frame = cap.read()
				if ret == False:
					break
        #print(frame.shape)
				frame=cv2.resize(frame, (256,224))
        #input_tensor = preprocess(frame)
        #print(frame.shape)
				if(i==0):
               
					data=frame
				else:
					data=np.concatenate((data,frame),axis=2)
			i+=1
			cap.release()
			cv2.destroyAllWindows()

		print(data.shape)
		input_tensor = tens(data)
		input_batch = input_tensor.unsqueeze(0)
		if torch.cuda.is_available():
			input_batch = input_batch.to('cuda')
			model.to('cuda')
		print(input_batch.shape)
		with torch.no_grad():
			output = model(input_batch)
	 

		labels = torch.tensor(int_labels)
		probabilities = torch.nn.functional.softmax(output[0], dim=0)
		print('probabilities',probabilities.size(0))
		
		top5_prob, top5_catid = torch.topk(probabilities, 5)
		print(torch.topk(probabilities, 5))
		if categories[top5_catid[0]] == current_name:
			trues.append(1)
			print('true')
		else:
			#trues.append(0)
			print('false')
		outputs.append(top5_prob[0].item())
    #print(top5_catid)
		#for i in range(top5_prob.size(0)):
		global res
		
		res = categories[top5_catid[0]]
	
		print(categories[top5_catid[0]], top5_prob[0].item())
		
   
    
   
	
		return render_template('index.html', filename=filename)
	
@app.route('/result',methods=['GET'])
def Index():
	
	print(res)
	return jsonify({'result': res})


@app.route('/display/<filename>')
def display_video(filename):
	#print('display_video filename: ' + filename)
	return redirect(url_for('static', filename='uploads/' + filename), code=301)

if __name__ == "__main__":
    app.run()








 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://134a-35-196-97-8.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [08/Jan/2023 16:35:16] "POST /upload HTTP/1.1" 200 -


(224, 256, 3)
size 3
input_tenor tensor([[[-1.9124, -1.9124, -1.9124,  ..., -1.9124, -1.8953, -1.8782],
         [-1.9124, -1.9124, -1.9124,  ..., -1.8782, -1.8439, -1.8953],
         [-1.8782, -1.9124, -1.9124,  ..., -1.8782, -1.8782, -1.8782],
         ...,
         [-1.7754, -1.7754, -1.7925,  ..., -1.7240, -1.7583, -1.7925],
         [-1.8268, -1.8439, -1.8097,  ..., -1.7754, -1.7583, -1.7412],
         [-1.7754, -1.7754, -1.7754,  ..., -1.7583, -1.7754, -1.7583]],

        [[-1.8782, -1.8953, -1.8953,  ..., -1.8782, -1.8953, -1.8782],
         [-1.8782, -1.8953, -1.8953,  ..., -1.8782, -1.8953, -1.8953],
         [-1.8953, -1.8953, -1.8953,  ..., -1.8953, -1.8782, -1.8782],
         ...,
         [-1.6727, -1.6727, -1.6898,  ..., -1.6727, -1.6555, -1.6727],
         [-1.6898, -1.6898, -1.6727,  ..., -1.6727, -1.6555, -1.6898],
         [-1.6727, -1.6727, -1.6727,  ..., -1.6555, -1.6727, -1.6555]],

        [[ 1.2557,  1.2899,  1.2899,  ...,  1.3755,  1.3584,  1.3755],
         [ 1

INFO:werkzeug:127.0.0.1 - - [08/Jan/2023 16:35:20] "GET /result HTTP/1.1" 200 -


ghazanfar
